# 🍒 Fruit Ninja Robot 🍑

## Program and Simulation Setup

In [ ]:
# Imports, imports everywhere
# Balance of Trade = Value of Exports - Value of Imports... - Gandhi, probably
import numpy as np
import time 
import threading
import random 
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors

from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    ConstantVectorSource,
    DiagramBuilder,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    MultibodyPlant,
    Parser,
    PiecewisePolynomial,
    PiecewiseQuaternionSlerp,
    RigidTransform,
    RollPitchYaw,
    RotationMatrix,
    Quaternion,
    Simulator,
    Solve,
    StartMeshcat,
    TrajectorySource,
    
    KinematicTrajectoryOptimization,
    BsplineTrajectory,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    PositionConstraint,
    OrientationConstraint,
    Role,
    
    Sphere,
    Rgba,
)

# from pydrake.geometry import DispatchLoadMessage, ConnectDrakeVisualizer
# from pydrake.systems.meshcat_visualizer import MeshcatPointCloudVisualizer

from pydrake.multibody import inverse_kinematics
from pydrake.trajectories import PiecewisePolynomial
from pydrake.perception import DepthImageToPointCloud

from manipulation.meshcat_utils import AddMeshcatTriad, PublishPositionTrajectory
from manipulation.scenarios import AddMultibodyTriad, AddIiwa, AddShape, AddWsg
from manipulation.utils import running_as_notebook
from manipulation.station import load_scenario, MakeHardwareStation, AddPointClouds

import time

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://f7bf94ae-48b6-4da0-b037-4d9acdec7e95.deepnoteproject.com/7000/


## Scenario Models Setup

In [ ]:
# Add elements to the scene. Not quite sure how this format works yet...
katana_filename = "/work/katana.sdf"
sphere_filename = "/work/sphere_modified.sdf"
red_sphere_filename = "/work/red_sphere.sdf"
green_sphere_filename = "/work/green_sphere.sdf"
blue_sphere_filename = "/work/blue_sphere.sdf"

fgp_x, fgp_y, fgp_z = 0, 2.5, 0.4
fruit_generation_point = [fgp_x, fgp_y, fgp_z]  # 5 meters in front of the robot, aligned horizontally and vertically
top_view_camera_position = [fgp_x, fgp_y/2, 2]  # Halfway between the robot and FGP, at a height of 2 meters
side_view_camera_position = [fgp_x-1, fgp_y/2, fgp_z]  # To the side, halfway between robot and FGP

# # put spheres in the camera's view to test
# spheres = [
#     {"name": "sphere_red", "color": Rgba(1, 0, 0, 1), "size": 0.05, "position": [fgp_x, fgp_y - 1, fgp_z]},
#     {"name": "sphere_green", "color": Rgba(0, 1, 0, 1), "size": 0.1, "position": [fgp_x, fgp_y - 2, fgp_z]},
#     {"name": "sphere_blue", "color": Rgba(0, 0, 1, 1), "size": 0.15, "position": [fgp_x, fgp_y - 3, fgp_z]}
# ]


scenario_data = f"""
directives:
- add_model:
    name: iiwa
    file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [0]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0

- add_model:
    name: katana
    file: file://{katana_filename}
- add_weld:   
    parent: iiwa::iiwa_link_7
    child: katana::katana
    X_PC:
        translation: [-0.34, 0.00, 0.05]
        rotation: !Rpy {{ deg: [0, -90, -90]}}

- add_model:
    name: half_black1
    file: file://{half_black1_filename}
- add_model:
    name: half_blue1
    file: file://{half_blue1_filename}
- add_model:
    name: half_green1
    file: file://{half_green1_filename}
- add_model:
    name: half_red1
    file: file://{half_red1_filename}
- add_model:
    name: half_yellow1
    file: file://{half_yellow1_filename}
- add_model:
    name: half_black2
    file: file://{half_black2_filename}
- add_model:
    name: half_blue2
    file: file://{half_blue2_filename}
- add_model:
    name: half_green2
    file: file://{half_green2_filename}
- add_model:
    name: half_red2
    file: file://{half_red2_filename}
- add_model:
    name: half_yellow2
    file: file://{half_yellow2_filename}
"""

# for sphere in spheres:
#     scenario_data += f"""
#     - add_model:
#         name: {sphere['name']}
#         file: package://drake/geometry/sphere.sdf
#     - add_weld:
#         parent: world
#         child: {sphere['name']}::link
#         X_PC:
#             translation: {sphere['position']}
#     """
    
# fruit cutting
# add weld to SDF file then remove weld on collision, plant.weld_frame() or plant.WeldFrames()
# https://deepnote.com/workspace/Manipulation-ac8201a1-470a-4c77-afd0-2cc45bc229ff/project/87d8044f-3eb2-4f25-b52f-f51a5eca68b7/notebook/writing-28e7162dd109465396eb35c75dcdf6e6
# or create two halves and assign position of one half to other half

scenario = load_scenario(data=scenario_data)

NameError: name 'half_black1_filename' is not defined

## Robot Setup

In [ ]:
# Create a wrapper class to control the IIWA arm.
class Ninja:
    def __init__(self, scenario, meshcat):
        """
        Initializate function robot ninja object. Honestly, this doesn't really do much...

        @param scenario: scene scenario string with object .sdf files.
        @param meshcat: meshcat object for simulation.
        """
        self.scenario = scenario
        self.meshcat = meshcat

        self.builder = None
        self.station = None
        self.plant = None
    
    def initialize_station(self):
        """
        Initialize manipulation station builder, station, plant, visualizer and other Drake infrastructure-related 
        items I don't understand at all. (Sun-Tzu 1864)
        """
        self.builder = DiagramBuilder()
        self.station = self.builder.AddSystem(MakeHardwareStation(self.scenario))
        self.plant = self.station.GetSubsystemByName("plant")
        self.scene_graph = self.station.GetSubsystemByName("scene_graph")
        # AddMultibodyTriad(self.plant.GetFrameByName("body"), self.scene_graph)
        AddMultibodyTriad(self.plant.GetFrameByName("katana"), self.scene_graph)

        # not needed due to having hardware station 
        self.visualizer = MeshcatVisualizer.AddToBuilder(
            self.builder,
            self.station.GetOutputPort("query_object"),
            self.meshcat,
            MeshcatVisualizerParams(delete_on_initialization_event=False),
        )


    def build_station(self, duration=0.01):
        """
        Build the manipulation station and extract contexts, bodies, and frames because reasons and you need to
        to use the optimization methods below.

        @param duration: timestamp to advance the simulator to.
        """
        self.diagram = self.builder.Build()

        self.diagram_context = self.diagram.CreateDefaultContext()
        self.plant_context = self.plant.CreateDefaultContext()
        self.katana = self.plant.GetBodyByName("katana")
        self.world_frame = self.plant.world_frame()
        self.katana_frame = self.plant.GetFrameByName("katana")

        self.initial_pose = self.plant.EvalBodyPoseInWorld(self.plant_context, self.katana)

        self.simulator = Simulator(self.diagram)
        self.simulator.set_target_realtime_rate(1.0)
        self.simulator.AdvanceTo(duration)

        self.visualizer.StartRecording(False)
        self.visualizer.PublishRecording()
    
    def setup_manipulation_station(self, setup_callback=None, duration=0.01):
        """
        Parent function to initialize and build the manipulation station.

        @param setup_callback: function to call before building the manipulation station, usually to connect input
                               and output ports, though idk how to make this work tbh.
        @param duration: timestamp to advance the simulator to.
        """
        self.initialize_station()
        self.build_station(duration)

    def create_kinematic_trajectory(self, trajectory, num_control_points=10, traj_duration_bounds=[0.5,5], max_tries=5):
        """
        Create a trajectory that contains all the RigidTransforms in trajectory through 
        KinematicTrajectoryOptimization. 

        @param trajectory: dictionary of rigid transforms and orientation boolans, 
               trajectory["poses"][i] should contain keyframe X_WG at index i, and
               trajectory["orientation_constraints"][i] should contain True if keyframe 
               X_WG needs an orientation constraint.
        @param num_control_points: number of points in the trajectory, I think...
        @param traj_duration_bounds: upper and lower limit for how long following the trajectory should take.
        @param max_tries: maximum number of attempts to solve mathematical program
        """
        # set up the optimization problem and add general costs and constraints for 
        # time, path length, etc.
        self.plant_root_context = self.plant.GetMyContextFromRoot(self.diagram_context)
        num_q = self.plant.num_positions()
        q0 = self.plant.GetPositions(self.plant_root_context)

        print(num_q, q0, num_control_points)

        trajopt = KinematicTrajectoryOptimization(num_q, num_control_points)
        prog = trajopt.get_mutable_prog()

        trajopt.AddDurationCost(1.0)
        trajopt.AddPathLengthCost(1.0)
        trajopt.AddPositionBounds(self.plant.GetPositionLowerLimits(), self.plant.GetPositionUpperLimits())
        trajopt.AddDurationConstraint(traj_duration_bounds[0], traj_duration_bounds[1])

        cut_pose_indices = []
        
        # for every rigid transform in trajectory["poses"], add a position constraint and cost so that the solver
        # tries to move the trajectory through each pose.
        for i in range(len(trajectory["poses"])):

            # transform each pose by length of katana along particular axis
            pose = trajectory["poses"][i]
            pos_constraint = PositionConstraint(
                self.plant,
                self.world_frame,
                pose.translation() - np.array([0.5, 0.01, 0.01]),
                pose.translation() + np.array([0.3, 0.01, 0.01]),
                self.katana_frame, 
                [0.0, 0.0, 0.0], # might change to allow for any section of the blade to hit point
                self.plant_root_context
            )
            # maybe add velocity constraint for fruit point
            trajopt.AddPathPositionConstraint(pos_constraint, i/len(trajectory["poses"]))

            if trajectory["orientation_constraints"][i]:
                orientation_constraint = OrientationConstraint(
                    self.plant,
                    self.world_frame,
                    pose.rotation(),
                    self.katana_frame,
                    RotationMatrix(),
                    np.pi/24,
                    self.plant_root_context
                )
                trajopt.AddPathPositionConstraint(orientation_constraint, i/len(trajectory["poses"]))
                
            if trajectory["types"][i] == CUT_POSE:
                cut_pose_indices.append(i)

            prog.AddQuadraticErrorCost(np.eye(num_q), q0, trajopt.control_points()[:, i])
        
        # add constraints so the trajectory starts and ends with zero velocity hopefully.
        trajopt.AddPathVelocityConstraint(np.zeros((num_q, 1)), np.zeros((num_q, 1)), 0)
        trajopt.AddPathVelocityConstraint(np.zeros((num_q, 1)), np.zeros((num_q, 1)), 1)

        ik = inverse_kinematics.InverseKinematics(self.plant)
        q_nominal = np.array([0.0, 0.6, 0.0, -1.75, 0.0, 1.0, 0.0])
        q_variables = ik.q()
        ik_prog = ik.prog()

        ik_bounds = 0.005
        upper_bound = trajectory["poses"][cut_pose_indices[0]].translation() + [0, 0, ik_bounds]
        lower_bound = trajectory["poses"][cut_pose_indices[0]].translation() + [0, 0, ik_bounds]
        ik.AddPositionConstraint(frameA=self.world_frame, frameB=self.katana_frame, p_BQ=np.zeros(3), 
                                 p_AQ_lower=lower_bound, p_AQ_upper=upper_bound)
        ik_prog.SetInitialGuess(q_variables, q_nominal)
        ik_result = Solve(ik_prog)
        ik_success = ik_result.is_success()
        
        if ik_success:
            q_guess = np.tile(q0.reshape((num_q, 1)), (1, trajopt.num_control_points()))
            q_guess[:, cut_pose_indices[0]] = ik_result.GetSolution()
            path_guess = BsplineTrajectory(trajopt.basis(), q_guess)
            trajopt.SetInitialGuess(path_guess)

        success = False
        for count in range(max_tries):
            # Compute a random initial guess here

            result = Solve(prog)
            success = result.is_success()
            if success:
                # trajopt.SetInitialGuess(trajopt.ReconstructTrajectory(result))
                PublishPositionTrajectory(trajopt.ReconstructTrajectory(result), self.diagram_context, self.plant, self.visualizer)
                break
            else:
                print(f"{count}: Failed: {result.get_solver_id().name()}...")
                # infeasible_constraints = result.GetInfeasibleConstraints(prog)
                # for c in infeasible_constraints:
                #     print(f"Infeasible constraint: {c}")
        # prog.SetOptions()
        # assert success

IndentationError: unexpected indent (3360656607.py, line 35)

In [ ]:
START_POSE_COLOR = Rgba(0.9, 0.1, 0.1, 0.5)
END_POSE_COLOR = Rgba(0.1, 0.9, 0.1, 0.5)
PRE_CUT_POSE_COLOR = Rgba(0.1, 0.1, 0.9, 0.5)
CUT_POSE_COLOR = Rgba(0.9, 0.1, 0.9, 0.5)

START_POSE = "START_POSE"
END_POSE = "END_POSE"
PRE_CUT_POSE = "PRE_CUT_POSE"
CUT_POSE = "CUT_POSE"

def render_pose(pose_id, pose, color, show_triad=True):
    # if show_triad:
    #     AddMeshcatTriad(meshcat, path=pose_id, X_PT=pose, opacity=color.a())
        
    meshcat.SetObject(pose_id, Sphere(0.03), rgba=color)
    meshcat.SetTransform(pose_id, pose)


# initialize ninja robot.
ninja = Ninja(scenario, meshcat)
ninja.setup_manipulation_station()

# create sample trajectory and visualize it.
trajectory_duration = 3
num_trajectory_samples = 5

def create_cut_trajectory(cut_positions):
    trajectory = {
        "poses": [],
        "orientation_constraints": [],
        "types": []
    }

    if not len(cut_positions):
        return trajectory_duration

    t = 0
    p0 = np.array(ninja.initial_pose.translation())
    R0 = ninja.initial_pose.rotation()
    P0 = RigidTransform(R0, p0)
    
    trajectory["poses"].append(P0)
    trajectory["orientation_constraints"].append(True)
    trajectory["types"].append(START_POSE)
    # render_pose(str(t), P0, START_POSE_COLOR, show_triad=trajectory["orientation_constraints"][-1])
    t += 1

    slope = np.zeros(3)
    R = RotationMatrix()

    if len(cut_positions) == 1:
        p = np.array(cut_positions[0])
        # R = cut_positions[0].rotation()

        slope = p - p0
        flat_slope = np.array([slope[0] * 0.0, slope[1], slope[2]])
        slope = slope/np.linalg.norm(slope)
        flat_slope = flat_slope/np.linalg.norm(flat_slope)
        # alpha = -np.arctan2(slope[0], slope[1]) # angle between dx and dy
        # beta = -np.arctan2(slope[2], slope[1]) # angle between dy and dz
        # # euler_angles = [0.0, beta, np.pi/2 + alpha]
        # euler_angles = [0.0, 0.0, 0.0]

        # Calculate the normalized direction vectors
        # direction_vector = p - p0
        # direction_vector[0] = 0.0
        # normalized_direction = direction_vector / np.linalg.norm(direction_vector)

        # Assuming the initial orientation is along the x-axis
        initial_orientation = RollPitchYaw(0, 0, 0)  # No rotation
        initial_rotation_matrix = initial_orientation.ToRotationMatrix()
        # quaternion_initial = Quaternion(initial_rotation_matrix)

        # Compute the target orientation based on the direction vector
        # This might involve calculating roll, pitch, and yaw based on the direction vector
        # The exact method depends on how you define the orientation in your problem
        # Assuming roll is zero or set based on your application's needs
        roll = int(p0[1] > p[1]) * np.pi
        pitch = np.arctan2(-slope[2], np.sqrt(slope[0]**2 + slope[1]**2))
        yaw = np.arctan2(slope[1], slope[0])
        target_orientation = RollPitchYaw(roll, pitch, yaw)
        target_rotation_matrix = target_orientation.ToRotationMatrix()
        
        R = initial_rotation_matrix.multiply(target_rotation_matrix)
        
        flat_roll = int(p0[1] > p[1]) * np.pi
        flat_pitch = np.arctan2(-flat_slope[2], np.sqrt(flat_slope[0]**2 + flat_slope[1]**2))
        flat_yaw = np.arctan2(flat_slope[1], flat_slope[0])
        flat_target_orientation = RollPitchYaw(flat_roll, flat_pitch, flat_yaw)
        flat_target_rotation_matrix = flat_target_orientation.ToRotationMatrix()
        
        flat_R = initial_rotation_matrix.multiply(flat_target_rotation_matrix)

        # combined_quaternion = quaternion_initial.multiply(quaternion_alignment)
        # R = RotationMatrix(combined_quaternion)

        p1 = p - slope * 0.1
        R1 = R
        P1 = RigidTransform(R1, p1)

        # trajectory["poses"].append(P1)
        # trajectory["orientation_constraints"].append(True)
        # trajectory["types"].append(PRE_CUT_POSE)
        # render_pose(str(t), P1, PRE_CUT_POSE_COLOR, show_triad=trajectory["orientation_constraints"][-1])
        # t += 1
    elif len(cut_positions) == 2:
        pass
    else:
        pass

    P = RigidTransform(R, cut_positions[0])
    for pos in cut_positions:
        P = RigidTransform(flat_R, pos)

        trajectory["poses"].append(P)
        trajectory["orientation_constraints"].append(True)
        trajectory["types"].append(CUT_POSE)
        render_pose(str(t), P, CUT_POSE_COLOR, show_triad=trajectory["orientation_constraints"][-1])
        t += 1

    finish_pose_dist = 0.1
    pf = np.array(P.translation()) + flat_slope * finish_pose_dist
    Rf = flat_R
    Pf = RigidTransform(Rf, pf)


    trajectory["poses"].append(Pf)
    trajectory["orientation_constraints"].append(True)
    trajectory["types"].append(END_POSE)
    # render_pose(str(t), Pf, END_POSE_COLOR, show_triad=trajectory["orientation_constraints"][-1])
    t += 1
    
    return trajectory


# trajectory = create_test_trajectory(trajectory_duration, num_trajectory_samples)
# trajectory = create_test_cut_trajectory(trajectory_duration, num_trajectory_samples)

# p1 = np.array([0.8, 0.4, 0.7])
p1 = np.array([0.75286486, 0.26899751, 0.78262947])
# R1 = RotationMatrix().MakeZRotation(np.pi/2)
# P1 = RigidTransform(R1, p1)

# trajectory = create_cut_trajectory([p1])
# print(len(trajectory["poses"]))

INFO:drake:Meshcat listening for connections at http://localhost:7001
==== LCM Warning ===
LCM detected that large packets are being received, but the kernel UDP
receive buffer is very small.  The possibility of dropping packets due to
insufficient buffer space is very high.

For more information, visit:
   http://lcm-proj.github.io/lcm/multicast_setup.html



In [ ]:
ninja.create_kinematic_trajectory(trajectory, num_trajectory_samples, [0.25, 5], 3)
# ninja.create_ik_trajectory(trajectory, num_trajectory_samples)

NameError: name 'trajectory' is not defined

In [ ]:
# initialize ninja robot.
ninja = Ninja(scenario, meshcat)
ninja.setup_manipulation_station()

# create sample trajectory and visualize it.
trajectory_duration = 3

for i in range(20):
    ninja.diagram.SetDefaultContext(ninja.diagram_context)
    p0 = np.array(ninja.initial_pose.translation())

    x = np.random.uniform(-0.1, 0.1)
    y = np.random.uniform(0.2, 0.5) * (1 if np.random.random() < 0.5 else -1)
    z = np.random.uniform(-0.1, 0.1)

    lower_bound = 0.5
    if i < 7:
        lower_bound = 1.5
    elif i < 12:
        lower_bound = 1.0
    elif i < 17:
        lower_bound = 0.5
    elif i < 20:
        loewr_bound = 1.0
    p = p0 + np.array([x, y, z])
    print(f"Trial {i+1}/20: Cut at point {p}!")
    trajectory = create_cut_trajectory([p])
    ninja.create_kinematic_trajectory(trajectory, num_trajectory_samples, [lower_bound, 5], 3)

    time.sleep(3)

INFO:drake:Meshcat listening for connections at http://localhost:7002
==== LCM Warning ===
LCM detected that large packets are being received, but the kernel UDP
receive buffer is very small.  The possibility of dropping packets due to
insufficient buffer space is very high.

For more information, visit:
   http://lcm-proj.github.io/lcm/multicast_setup.html

Trial 1/20: Cut at point [0.71881491 0.23584878 0.89929316]!
7 [ 0.   0.1  0.  -1.2  0.   1.6  0. ] 5
Trial 2/20: Cut at point [ 0.83428404 -0.24982845  0.84434786]!
7 [ 0.   0.1  0.  -1.2  0.   1.6  0. ] 5
Trial 3/20: Cut at point [0.70556416 0.44572583 0.72260922]!
7 [ 0.   0.1  0.  -1.2  0.   1.6  0. ] 5
Trial 4/20: Cut at point [0.70908929 0.20112393 0.77894199]!
7 [ 0.   0.1  0.  -1.2  0.   1.6  0. ] 5
Trial 5/20: Cut at point [0.786908   0.47838092 0.72238184]!
7 [ 0.   0.1  0.  -1.2  0.   1.6  0. ] 5
Trial 6/20: Cut at point [0.86736263 0.3942739  0.84397711]!
7 [ 0.   0.1  0.  -1.2  0.   1.6  0. ] 5
Trial 7/20: Cut at point

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f7bf94ae-48b6-4da0-b037-4d9acdec7e95' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>